## [1] 데이터 로더

In [1]:
import numpy as np
import torch
import torch.optim as optim


torch.manual_seed(1)

! curl 'https://raw.githubusercontent.com/sejongresearch/2020.Spring.AI/master/HW/zoo_class.csv' -o 'zoo_class.csv'
print('Done..\n')

xy=np.loadtxt('zoo_class.csv',delimiter=',', dtype=np.float32)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3535  100  3535    0     0  22660      0 --:--:-- --:--:-- --:--:-- 22660
Done..



In [2]:
x_data=xy[:, 0:-1]
y_data=xy[:,[-1]].squeeze()    # -전체 행의 1열만
                                # y_data=xy[:,-1]

x_train = torch.FloatTensor(x_data)  
y_train = torch.LongTensor(y_data)  

print(x_train[:5])   
print(y_train[:5])

print(x_train.shape)  
print(y_train.shape)    

tensor([[1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 4., 1., 0., 1.],
        [0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 1., 0., 1.]])
tensor([0, 0, 3, 0, 0])
torch.Size([101, 16])
torch.Size([101])


In [3]:
y_train.max()  #라벨 종류의 개수를 확인 가능
    
# max= 6이므로. 0~6 총 7개임을 알 수 있다.

tensor(6)

## [2] 모델 학습

In [4]:
import torch.nn.functional as F  # for softmax

# 모델 초기화
# feature 16개, 7개의 클래스

nb_class = 7
nb_data = len(y_train)

W = torch.zeros((16, nb_class), requires_grad=True)  
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)   #lr=0.01

nb_epochs = 10000                     #epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
           # softmax함수를 호출하거나, cross-entropy를 그대로 써도 됌.
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    
    # cost 표현번 1번 예시
      # one-hot encoding.
    y_one_hot = torch.zeros(nb_data, nb_class)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()


    # cost로 H(x) 개선 (w 업데이트)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

  #scatter() 할 때, unsqueeze()는 하지 않음. 이미 클래스의 값이 각 리스트

# lr=10,  에폭 10000 Cost: 1.177567에서 감소하지 않음
# lr=10,  에폭 1000 Cost: 1.177567에서 감소하지 않음

# --> lr=1, 에폭 10000 Cost: 1.167862

Epoch    0/10000 Cost: 1.945910
Epoch 1000/10000 Cost: 1.295598
Epoch 2000/10000 Cost: 1.235436
Epoch 3000/10000 Cost: 1.209874
Epoch 4000/10000 Cost: 1.207238
Epoch 5000/10000 Cost: 1.206057
Epoch 6000/10000 Cost: 1.205236
Epoch 7000/10000 Cost: 1.170907
Epoch 8000/10000 Cost: 1.169181
Epoch 9000/10000 Cost: 1.168367
Epoch 10000/10000 Cost: 1.167862


##[3] 정답 예측

In [5]:
# 학습된 W,b를 통한 클래스 예측

hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1) 
          # torch.argmax(): hypothesis(=wx)에서 가장 큰 값의 인덱스를 return


#print(hypothesis) # 범주형데이터로 바뀌기 전. wx를 구한 상태.
print(predict)    # 예측한 클래스 
print(y_train)    # 실제 데이터.

# 정확도 계산 
correct_prediction = predict.float() == y_train
print(correct_prediction)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

# 정확도는 100%

tensor([0, 0, 3, 0, 0, 0, 0, 3, 3, 0, 0, 1, 3, 6, 6, 6, 1, 0, 3, 0, 1, 1, 0, 1,
        5, 4, 4, 0, 0, 0, 5, 0, 0, 1, 3, 0, 0, 1, 3, 5, 5, 1, 5, 1, 0, 0, 6, 0,
        0, 0, 0, 5, 4, 6, 0, 0, 1, 1, 1, 1, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        6, 3, 0, 0, 2, 6, 1, 1, 2, 6, 3, 1, 0, 6, 3, 1, 5, 4, 2, 2, 3, 0, 0, 1,
        0, 5, 0, 6, 1], grad_fn=<NotImplemented>)
tensor([0, 0, 3, 0, 0, 0, 0, 3, 3, 0, 0, 1, 3, 6, 6, 6, 1, 0, 3, 0, 1, 1, 0, 1,
        5, 4, 4, 0, 0, 0, 5, 0, 0, 1, 3, 0, 0, 1, 3, 5, 5, 1, 5, 1, 0, 0, 6, 0,
        0, 0, 0, 5, 4, 6, 0, 0, 1, 1, 1, 1, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        6, 3, 0, 0, 2, 6, 1, 1, 2, 6, 3, 1, 0, 6, 3, 1, 5, 4, 2, 2, 3, 0, 0, 1,
        0, 5, 0, 6, 1])
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, Tr